# import

In [4]:
%load_ext autoreload

%autoreload 2
import time

import pandas
import argparse

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions


from chainer.datasets import tuple_dataset
from chainer.dataset import convert, concat_examples

import numpy
import os
import glob
import pandas
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# define of model

In [5]:
class MLP(chainer.Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)# n_units -> n_out
            self.l3 = L.Linear(None, n_out)
            
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

# preparation of dataset
## import dataset

In [3]:
co_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/train_data/2018.07.10/cotton-*"))
sl_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/train_data/2018.07.10/silk-*"))
li_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/train_data/2018.07.10/linen-*"))
wo_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/train_data/2018.07.10/wool-*"))
ac_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/train_data/2018.07.10/acrylic-*"))
ny_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/train_data/2018.07.10/nylon-*"))
ra_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/train_data/2018.07.10/rayon-*"))
po_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/train_data/2018.07.10/poly-*"))
pp_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/train_data/2018.07.10/pp-*"))

tco_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/test_data/2018.07.10/cotton-*"))
tsl_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/test_data/2018.07.10/silk-*"))
tli_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/test_data/2018.07.10/linen-*"))
two_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/test_data/2018.07.10/wool-*"))
tac_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/test_data/2018.07.10/acrylic-*"))
tny_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/test_data/2018.07.10/nylon-*"))
tra_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/test_data/2018.07.10/rayon-*"))
tpo_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/test_data/2018.07.10/poly-*"))
tpp_list = sorted(glob.glob("C:/Users/Owner/Desktop/NN_project/test_data/2018.07.10/pp-*"))


for i, fname in enumerate(co_list):
#     print(r"%d: %s" % (i, fname))
    xls = pandas.read_csv(fname, skiprows=4, delimiter="\t", header=None)
    name = os.path.split(fname)[1].split(".")[0] # add:: 2018.07.23

    if i == 0:       
        arr = xls.as_matrix()[:,1]  # numpy.array
        hddr = name # add:: 2018.07.23
    else:
        arr = numpy.vstack((arr, xls.as_matrix()[:,1]))
        hddr = numpy.hstack((hddr, name)) # add:: 2018.07.23

print(arr.shape)        

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


(7, 2385)


In [58]:
import pandas
f = pandas.read_excel("C:/Users/Owner/Desktop/blood_glucose.xls")

In [59]:
arr = f[:].T
arr = arr.astype(numpy.float32)

In [60]:
print(arr.shape)

(60, 1001)


In [46]:
label = f[1000:].T
label = label.astype(numpy.int32)
print(label.dtype)

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [63]:
label = numpy.hstack((numpy.zeros(7), numpy.ones(9), numpy.ones(14)*2, numpy.ones(14)*3, numpy.ones(6)*4,
                    numpy.zeros(2), numpy.ones(2), numpy.ones(2)*2, numpy.ones(2)*3, numpy.ones(2)*4)).astype(numpy.int32)
print(label)
print(label.shape)

[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 0 0 1 1 2 2 3 3 4 4]
(60,)


In [4]:
for list_ in (sl_list, li_list, wo_list, ac_list, ny_list, ra_list, pp_list,po_list, 
              tco_list, tsl_list, tli_list, two_list, tac_list, tny_list, tra_list, tpp_list, tpo_list):
    for i, fname in enumerate(list_):
        xls = pandas.read_csv(fname, skiprows=4, delimiter="\t", header=None)
        arr = numpy.vstack((arr, xls.as_matrix()[:,1]))
        hddr = numpy.hstack((hddr, os.path.split(fname)[1].split(".")[0])) # add:: 2018.07.23
    print(arr.shape)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


(14, 2385)
(21, 2385)
(28, 2385)
(35, 2385)
(42, 2385)
(49, 2385)
(56, 2385)
(63, 2385)
(66, 2385)
(69, 2385)
(72, 2385)
(75, 2385)
(78, 2385)
(81, 2385)
(84, 2385)
(87, 2385)
(90, 2385)


In [44]:
label = numpy.hstack((numpy.zeros(7), numpy.ones(7), numpy.ones(7)*2, numpy.ones(7)*3, numpy.ones(7)*4, 
                      numpy.ones(7)*5, numpy.ones(7)*6, numpy.ones(7)*7, numpy.ones(7)*8,
                      numpy.zeros(3),numpy.ones(3)*1, numpy.ones(3)*2, 
                      numpy.ones(3)*3,numpy.ones(3)*4, 
                      numpy.ones(3)*5, numpy.ones(3)*6, numpy.ones(3)*7, numpy.ones(3)*8)).astype(numpy.int32)
print(label.dtype)

int32


In [6]:
label

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3,
       3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6,
       6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 0, 0, 0,
       1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8,
       8, 8])

In [7]:
arr = arr.astype(numpy.float32)  # float64 ==> float32
print(arr.shape)

(90, 2385)


## generate of chainer.iterator

In [64]:
# 解析データの準備
id_train = label[0:50]
id_test = label[50:60]

train = tuple_dataset.TupleDataset(arr[id_train],id_train.T)
train_iter = chainer.iterators.SerialIterator(train, 50)

test = tuple_dataset.TupleDataset(arr[id_test], id_test.T)
test_iter = chainer.iterators.SerialIterator(test, 10, repeat=False, shuffle=False)

KeyError: '[0 0 0 0 0 0 0] not in index'

# setup of model

In [9]:
net = MLP(1000, 9)
model = L.Classifier(net)


In [10]:
# Setup an optimizer
#optimizer = chainer.optimizers.Adam(alpha=0.001, beta1=0.9, beta2=0.999, eps=1e-08)
optimizer = chainer.optimizers.SGD()
optimizer.setup(model)
# help(chainer.optimizers.adam)


# train

In [11]:
epoch = 20
train_iter.reset()
out_directory = "./Result2018_oono/%s" % time.strftime("%Y%m%d_%H%M%S")
# test_iter.reset()

# Set up a trainer
updater = training.updaters.StandardUpdater(
    train_iter, optimizer)
trainer = training.Trainer(updater, (epoch, 'epoch'), out=out_directory)

# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model))

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot for each specified epoch
# frequency = args.epoch if args.frequency == -1 else max(1, args.frequency)
# trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

# Save two plot images to the result dir
# if args_plot and extensions.PlotReport.available():
trainer.extend(
    extensions.PlotReport(['main/loss', 'validation/main/loss'],
                          'epoch', file_name='loss.png'))
trainer.extend(
    extensions.PlotReport(
        ['main/accuracy', 'validation/main/accuracy'],
        'epoch', file_name='accuracy.png'))

# Print selected entries of the log to stdout
# Here "main" refers to the target link of the "main" optimizer again, and
# "validation" refers to the default name of the Evaluator extension.
# Entries other than 'epoch' are reported by the Classifier link, called by
# either the updater or the evaluator.
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

# Print a progress bar to stdout
# trainer.extend(extensions.ProgressBar())

# chainer.serializers.load_npz("", trainer)

In [12]:
trainer.run()


epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           2.20242     2.20074               0.111111       0.111111                  0.19084       
2           2.20074     2.19914               0.111111       0.111111                  0.746339      
3           2.19914     2.19756               0.111111       0.111111                  1.07496       
4           2.19756     2.196                 0.111111       0.111111                  1.47299       
5           2.196       2.19447               0.111111       0.111111                  1.81265       
6           2.19448     2.19297               0.111111       0.111111                  2.17654       
7           2.19297     2.19148               0.111111       0.111111                  2.48442       
8           2.19148     2.19001               0.111111       0.111111                  2.83321       
9           2.19001     2.18858               0.111111       0.222222               

In [15]:
#import h5py
#net.to_cpu()

#chainer.serializers.save_hdf5("C:/Users/Owner/Desktop/Result2018_oono/%s/model.hdf5" % time.strftime("%Y%m%d_%H%M%S"), obj=model)

#numpy.save(arr=id_test, file="C:/Users/Owner/Desktop/Result2018_oono/%s/test_id.npy" % time.strftime("%Y%m%d_%H%M%S"))
#numpy.save(arr=id_train, file="C:/Users/Owner/Desktop/Result2018_oono/%s/train_id.npy"  % time.strftime("%Y%m%d_%H%M%S"))

#net = model.predictor
#cnt = 0
#for i in range(33):
    #print("label: %d, pred: %d [%s]"% (id_test[i], numpy.argmax(net(arr[i].reshape(1, -1)).data),
                                     #id_test[i] == numpy.argmax(net(arr[i].reshape(1, -1)).data)))
    #if id_test.T[i] == numpy.argmax(net(arr[i].reshape(1, -1)).data):
        #cnt = cnt + 1
#print("%1.1f [%%]" % (float(cnt/33*100)))

In [14]:
test_iter.reset()

test_batch = test_iter.next()
test_spc, test_ref = concat_examples(test_batch)  # Test Dataset
for i in range(27):
    ref = test_ref[i]
    pred = numpy.argmax(net(test_spc[i].reshape(1, -1)).data)
    print("Ref: %d, Pred: %d %s" % (ref, pred, ref == pred))


Ref: 0, Pred: 0 True
Ref: 0, Pred: 0 True
Ref: 0, Pred: 0 True
Ref: 1, Pred: 0 False
Ref: 1, Pred: 0 False
Ref: 1, Pred: 0 False
Ref: 2, Pred: 0 False
Ref: 2, Pred: 0 False
Ref: 2, Pred: 0 False
Ref: 3, Pred: 0 False
Ref: 3, Pred: 0 False
Ref: 3, Pred: 0 False
Ref: 4, Pred: 0 False
Ref: 4, Pred: 0 False
Ref: 4, Pred: 0 False
Ref: 5, Pred: 0 False
Ref: 5, Pred: 0 False
Ref: 5, Pred: 0 False
Ref: 6, Pred: 0 False
Ref: 6, Pred: 0 False
Ref: 6, Pred: 0 False
Ref: 7, Pred: 7 True
Ref: 7, Pred: 7 True
Ref: 7, Pred: 7 True
Ref: 8, Pred: 7 False
Ref: 8, Pred: 7 False
Ref: 8, Pred: 7 False
